<a href="https://colab.research.google.com/github/DonErnesto/masterclassSFI_2021/blob/main/notebooks/CreditCardUnsupervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fraud detection with Unsupervised Machine Learning

**Introduction**


The purpose of this Jupyter notebook is to take you through several unsupervised outlier detection algorithms, and show their value for the purpose of fraud detection. The completely different approach compared to the supervised case, is that we assume we have insufficient labels to predict future patterns, as is often the case in fraud detection. In applying an outlier detection for fraud detection, we hypothesize that payment patterns that are untypical are more likely to be fraudulent. If this hypothesis is true, a compliance function is much more likely to spot fraudulent payments when inspecting a sample of payments that are outliers, compared to a sample that is drawn at random. 


Generally, whenever an unsupervised approach is chosen, there are no labels available; neither for algorithm optimization, nor for comparison or validation. For this masterclass, we do use a labeled dataset, which will be useful for us to get a feeling of how the various algorithms perform and compare. 


The data was taken from https://www.kaggle.com/mlg-ulb/creditcardfraud, and downsampled for the purpose of this masterclass. 


In [ ]:
## Data import from Github
import os
if not os.path.exists('X_unsupervised.csv.zip'):
    !curl -O https://raw.githubusercontent.com/DonErnesto/masterclassSFI_2021/main/data/X_unsupervised.csv.zip

We will be using the "pandas" package for data handling and manipulation, and later "scikit-learn" (imported with "sklearn") for various outlier detection algorithms. 

In [ ]:
## Package import: pandas for data handling and manipulation
import pandas as pd

# A small hack: "monkey-patching" the DataFrame class to add column-wise normalization as a method
def normalize_columns(self,):
    return (self - self.mean()) / self.std()

pd.DataFrame.normalize_columns = normalize_columns

Next, we will load the data in a so-called DataFrame (a pandas object), and inspect it by plotting the N-top rows

In [ ]:
X = pd.read_csv('X_unsupervised.csv.zip')
# .head() returns a DataFrame, that consists of the first N (default: N=5) rows 
# of the DataFrame it is applied on
X.head() 

The data describes credit card transactions, one transaction per row. 

As you may notice, all features are numeric. All Vx features are the result of a mathematical operation called PCA. In reality, we have to deal often with non-numerical (for instance, categorical data), that requires some effort to make it numerical and suitable for the mathematical models we work with. 

The pre-fabricated data thus saves us considerable time. 

Let us first determine the dimensions of the DataFrame (note that the first dimension goes along the rows, the second along columns):

## Generating a homemade outlier score

Generate an array with outlier scores based on your own hand-made logic. Store the outlier predictions in a pandas Series with the name "homemade_outlier_scores", using the examples below. 


**Question:**

Below, there are several options to create a hand-crafted outlier score. 

Which one would you chose, and why? Uncomment your preference, to assign the outcome to the variable 'homemade_outlier_scores'. If you see a better solution, you are free to implement that. 


In [ ]:
# Some examples to make an outlier score below. Uncomment (remove the "#") to execute it.

# homemade_outlier_scores = X['V1'].abs()
# homemade_outlier_scores = (X.normalize_columns()**2).mean(axis=1)
# homemade_outlier_scores = X['Amount']
# homemade_outlier_scores = X.drop(columns='Time').abs().max(axis=1) # .drop() returns the cropped dataframe



In [ ]:
# To verify the shape, add .shape to the dataframe and look at the output. What shape should it be?
homemade_outlier_scores.shape

## Use an outlier algorithm to generate outlier scores (10 minutes)

Go to the section of the outlier algorithm assigned to you or chosen by you to generate your scores. 


In [ ]:
# from sklearn.neighbors import LocalOutlierFactor
# !pip install seaborn==0.11.1 # Needed for plotting
# !pip install tensorflow
!pip install pyod
import numpy as np
from sklearn.covariance import EmpiricalCovariance
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import IsolationForest
from sklearn.mixture import GaussianMixture
from pyod.models.auto_encoder import AutoEncoder
from sklearn.preprocessing import MinMaxScaler


### Mahalanobis Distance

As the name suggests, the Mahalanobis distance is a distance-based outlier detection. The Mahalonobis distance is a generalization of the distance in standard deviation to any multidimensional, normal distribution.

In the cells below: 
- Create an EmpiricalCovariance object with the correct parameters (find out which ones)
- Fit the data to this model
- Assign the scores to "mah_outlier_scores", using the method "mahalanobis"

Can the outcome of this method be directly used as an outlier score?

In [ ]:
?EmpiricalCovariance

# cov = EmpiricalCovariance(xxxx)
# cov.fit(xxxx)
# mah_outlier_scores = cov.???

**Question**: in which situation is the Mahalonobis-distance equal to do a simple, column-wise mean? 

### Nearest neighbours

The family ofnearest neighbours-based algorithms looks at the immediate "neighbour" in terms of nearest points. 
If a point is far-removed from its neighbours, it may be considered untypical. The distance may however be determined in several different ways. 

In the cells below: 
- Create an EmpiricalCovariance object with the correct parameters (find out which ones)
- Fit the data to this model
- Assign the scores to "mah_outlier_scores", using the method "mahalonobis"

Let's create a NearestNeighbors object, and use that. First, we may want to read some documentation regarding the NearestNeighbors class:

In [ ]:
?NearestNeighbors
# nn = NearestNeighbors(xxxxx)
# nn.fit(xxxxx)
# distances_to_neighbors = nn.kneighbors()[0]

The "heavy lifting" was done by the kneighbors() method. 
It returns the distances to the first N points, and the index of the nearest point 


As a final step, reduce the distance matrix (size: N points x N neighbours) to scores, one per point (row). 
Use an appropiate numpy function (ideas are: mean, median, min, max)

In [ ]:
# knn_mean_outlier_scores = np.xxxx(distances_to_neighbors, axis=1)

**Question**: what is an interpretation (say when n_neighbours is 11) for the

- median
- min
- max

distance to the n_neighbours?

### Isolation Forest algorithm

The isolation forest algorithm measures how difficult it is to isolate a point from the rest of the data, using a random splitting algorithm many times. We want to have 1000 estimators (trees), and each estimator to be based on a sample of 1024 points

In the cells below: 
- Create an IsolationForest object with the correct parameters
- Fit the IsolationForest object with the data
- Get the scores using .score_samples()


In [ ]:
?IsolationForest
# iforest = IsolationForest(xxxx=xxxxx, xxx=xxxxxx)
# iforest.fit(xxxx)


The score as returned by score_samples() is a measure for the number of needed splits to isolate a point. 

**Question:** Is a high score or a low score an indication for a point being an outlier? 
Reflect this in your score calculation. 


In [ ]:
# iforest_outlier_scores = iforest.score_samples(xxx)

### Gaussian Mixture

The Gaussian Mixture assumes the data consists of one or multiple "blobs" of clusters with some normal distribution (NB: with a co-variance matrix constrained to be spherical, diagonal or non-constrained - full). 
After fitting, the method .score_samples() returns some probability measure (probability density of the point within the gaussian mixture distribution). 

In the cells below, 

- Create a GaussianMixture object with sensible parameters 
- Fit the object to the data
- Get scores for the individual data points using .score_samples() 

In [ ]:
?GaussianMixture
# gmm = GaussianMixture(xxx=xxxx, xxx=xxx, random_state=1) 
# gmm.fit(xxxx)

**Question**: The .score_samples method returns a probability. Can the scores be used as they are or do they need to be modified? Reflect this in your calculation. 

In [ ]:
# gmm_scores = gmm.score_samples(xxx)


###  Autoencoder

Autoencoders are a special type of neural networks, that are trained to effectively compress and decompress a signal. The idea behind using these networks for outlier detection, is that the neural network is expected to handle "typical" datapoints well, whereas it will struggle with outliers. 

We use the pyod AutoEncoder class, because this way we don't have to define all details of the neural network architecture, but can specify the main parameters. 

- Use the scaled data X_scaled (why do you think?)
- 

In [ ]:
?AutoEncoder

In [ ]:
X_scaled = MinMaxScaler().fit_transform(X)
clf = AutoEncoder(
    hidden_neurons=[10, xxxx, 10], # Choose bottleneck here!
    hidden_activation='elu',
    output_activation='sigmoid', # Choose an activation ('linear', 'sigmoid', 'relu', 'elu' are some possibilities)
    optimizer='adam',
    epochs=10,
    batch_size=16,
    dropout_rate=0.0, #may not be needed here
    l2_regularizer=0.0,
    validation_size=0.1,
    preprocessing=False, #NB: this uses sklearn's StandardScaler
    verbose=1,
    random_state=1,
)

In [ ]:
clf.fit(X_scaled)

In [ ]:
?clf.decision_scores_

**Question:** Inspect the documentation on .decision_scores_, and calculate the outlier scores. 

In [ ]:
# autoenc_outlier_scores = clf.decision_scores_

##  3: Plot and compare results (5 min)

In the next section, you will compare how your algorithm did against your "home-made" algorithm, using the labels (ground-truth: is a point an outlier or not? In this case: is a transaction fraudulent or not?). 
Note that this information is usually not available for those problems where we decide to use outlier detection.


Look carefully at the plots and assess their meaning. 



In [ ]:
# Get the labels, and a helper module
!curl -O https://raw.githubusercontent.com/DonErnesto/masterclassSFI_2021/main/data/y_unsupervised.csv.zip
!curl -O https://raw.githubusercontent.com/DonErnesto/masterclassSFI_2021/main/ml_utils.py
y = pd.read_csv('y_unsupervised.csv.zip')['Class']


In [ ]:
from ml_utils import plot_top_N, plot_outlier_scores

### Show the conditional distributions of the scores, and the AUC metrics

(NB: only plot your "homemade" score and your own algorithm. 

In [ ]:
_ = plot_outlier_scores(y.values, np.log1p(homemade_outlier_scores), title='Homemade: ')

In [ ]:
_ = plot_outlier_scores(y.values, np.log1p(knn_mean_outlier_scores), title='KNN: ')

In [ ]:
_ = plot_outlier_scores(y.values, np.log1p(iforest_outlier_scores), title='Isolation Forest: ')

In [ ]:
_ = plot_outlier_scores(y.values, np.log10(mah_outlier_scores), title='Mahalonobis: ')

In [ ]:
_ = plot_outlier_scores(y.values, np.log10(autoenc_outlier_scores), title='Autoencoder: ')

In [ ]:
_ = plot_outlier_scores(y.values, np.log10(gmm_scores - np.min(gmm_scores) + 1), title='GMM: ')

### Showing the precision@top-N

In [ ]:
_ = plot_top_N(y_true=y, scores=homemade_outlier_scores, N=100)

In [ ]:
_ = plot_top_N(y_true=y, scores=knn_mean_outlier_scores, N=100)

In [ ]:
_ = plot_top_N(y_true=y, scores=iforest_outlier_scores, N=100)

In [ ]:
_ = plot_top_N(y_true=y, scores=mah_outlier_scores, N=100)

In [ ]:
_ = plot_top_N(y_true=y, scores=autoenc_outlier_scores, N=100)

In [ ]:
_ = plot_top_N(y_true=y, scores=gmm_scores, N=100)

**Question:** based on the number of positives, what precision do you expect when randomly guessing?

## Discussion



- How did you construct your home-made outlier model?
- How did it perform?
- What choices did you make for the outlier algorithm, if any, and why?
- How do both compare?

